In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration, DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
# Step 1: Load and Prepare the Dataset
df = pd.read_csv("/content/agricultural_land_clauses.csv")
df["label"] = df["label"].map({"SAFE": 0, "RISKY": 1})  # For risk analysis
dataset = Dataset.from_pandas(df)

In [ ]:
# Step 2: Fine-Tune BART for Summarization with LoRA
# Load tokenizer and model
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# Configure LoRA for BART
lora_config_bart = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # Sequence-to-sequence task (summarization)
    r=8,  # Rank of the low-rank matrices
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout for LoRA layers
    target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
)
bart_model = get_peft_model(bart_model, lora_config_bart)
print("LoRA applied to BART model")

LoRA applied to BART model


In [ ]:
# Tokenize the dataset for summarization
def preprocess_bart_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]
    model_inputs = bart_tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = bart_tokenizer(targets, max_length=150, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_bart_dataset = dataset.map(preprocess_bart_function, batched=True)
train_bart_dataset = tokenized_bart_dataset.train_test_split(test_size=0.1)["train"]
eval_bart_dataset = tokenized_bart_dataset.train_test_split(test_size=0.1)["test"]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments for BART
bart_training_args = TrainingArguments(
    output_dir="./fine_tuned_bart_lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs_bart",
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer for BART
bart_trainer = Trainer(
    model=bart_model,
    args=bart_training_args,
    train_dataset=train_bart_dataset,
    eval_dataset=eval_bart_dataset,
)

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Fine-tune BART
print("Fine-tuning BART with LoRA...")
bart_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Fine-tuning BART with LoRA...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: imayank45 (imayank45-svkm-s-narsee-monjee-institute-of-management-s) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
10,10.417600,10.284489
20,10.272500,10.268735
30,10.251800,10.241377
40,10.312100,10.201578
50,10.387300,10.147865
60,10.157800,10.077930
70,10.107800,9.990453
80,10.127400,9.885183
90,10.063700,9.763935
100,9.838500,9.631842


Could not locate the best model at ./fine_tuned_bart_lora/checkpoint-670/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=675, training_loss=4.363753256621184, metrics={'train_runtime': 1045.5279, 'train_samples_per_second': 2.582, 'train_steps_per_second': 0.646, 'total_flos': 2935375685222400.0, 'train_loss': 4.363753256621184, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned BART model
bart_model.save_pretrained("./fine_tuned_bart_lora")
bart_tokenizer.save_pretrained("./fine_tuned_bart_lora")

('./fine_tuned_bart_lora/tokenizer_config.json',
 './fine_tuned_bart_lora/special_tokens_map.json',
 './fine_tuned_bart_lora/vocab.json',
 './fine_tuned_bart_lora/merges.txt',
 './fine_tuned_bart_lora/added_tokens.json')

In [ ]:
!zip -r fine_tuned_bart_lora.zip fine_tuned_bart_lora

  adding: fine_tuned_bart_lora/ (stored 0%)
  adding: fine_tuned_bart_lora/special_tokens_map.json (deflated 85%)
  adding: fine_tuned_bart_lora/merges.txt (deflated 53%)
  adding: fine_tuned_bart_lora/checkpoint-500/ (stored 0%)
  adding: fine_tuned_bart_lora/checkpoint-500/training_args.bin (deflated 52%)
  adding: fine_tuned_bart_lora/checkpoint-500/adapter_config.json (deflated 54%)
  adding: fine_tuned_bart_lora/checkpoint-500/optimizer.pt (deflated 7%)
  adding: fine_tuned_bart_lora/checkpoint-500/README.md (deflated 66%)
  adding: fine_tuned_bart_lora/checkpoint-500/scheduler.pt (deflated 57%)
  adding: fine_tuned_bart_lora/checkpoint-500/rng_state.pth (deflated 25%)
  adding: fine_tuned_bart_lora/checkpoint-500/adapter_model.safetensors (deflated 7%)
  adding: fine_tuned_bart_lora/checkpoint-500/trainer_state.json (deflated 83%)
  adding: fine_tuned_bart_lora/tokenizer_config.json (deflated 75%)
  adding: fine_tuned_bart_lora/adapter_config.json (deflated 54%)
  adding: fine_tu

In [ ]:
# Step 3: Fine-Tune DistilBERT for Risk Analysis with LoRA
# Load tokenizer and model
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
distilbert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Configure LoRA for DistilBERT
lora_config_distilbert = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification task (risk analysis)
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"],  # Apply LoRA to attention layers in DistilBERT
)
distilbert_model = get_peft_model(distilbert_model, lora_config_distilbert)
print("LoRA applied to DistilBERT model")

LoRA applied to DistilBERT model


In [ ]:
# Tokenize the dataset for risk analysis
def preprocess_distilbert_function(examples):
    model_inputs = distilbert_tokenizer(examples["text"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = examples["label"]
    return model_inputs

tokenized_distilbert_dataset = dataset.map(preprocess_distilbert_function, batched=True)
train_distilbert_dataset = tokenized_distilbert_dataset.train_test_split(test_size=0.1)["train"]
eval_distilbert_dataset = tokenized_distilbert_dataset.train_test_split(test_size=0.1)["test"]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments for DistilBERT
distilbert_training_args = TrainingArguments(
    output_dir="./fine_tuned_distilbert_lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs_distilbert",
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer for DistilBERT
distilbert_trainer = Trainer(
    model=distilbert_model,
    args=distilbert_training_args,
    train_dataset=train_distilbert_dataset,
    eval_dataset=eval_distilbert_dataset,
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Fine-tune DistilBERT
print("Fine-tuning DistilBERT with LoRA...")
distilbert_trainer.train()

Fine-tuning DistilBERT with LoRA...


Step,Training Loss,Validation Loss
10,0.698400,0.693896
20,0.694200,0.692714
30,0.696700,0.690803
40,0.684100,0.687997
50,0.683100,0.684377
60,0.680700,0.680167
70,0.680200,0.674976
80,0.680400,0.669351
90,0.665400,0.662815
100,0.667500,0.655650


Could not locate the best model at ./fine_tuned_distilbert_lora/checkpoint-670/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=675, training_loss=0.2239943719657, metrics={'train_runtime': 206.669, 'train_samples_per_second': 13.064, 'train_steps_per_second': 3.266, 'total_flos': 363796398489600.0, 'train_loss': 0.2239943719657, 'epoch': 3.0})

In [ ]:
# Save the fine-tuned DistilBERT model
distilbert_model.save_pretrained("./fine_tuned_distilbert_lora")
distilbert_tokenizer.save_pretrained("./fine_tuned_distilbert_lora")

('./fine_tuned_distilbert_lora/tokenizer_config.json',
 './fine_tuned_distilbert_lora/special_tokens_map.json',
 './fine_tuned_distilbert_lora/vocab.txt',
 './fine_tuned_distilbert_lora/added_tokens.json')

In [ ]:
!zip -r fine_tuned_distilbert_lora.zip fine_tuned_distilbert_lora

  adding: fine_tuned_distilbert_lora/ (stored 0%)
  adding: fine_tuned_distilbert_lora/special_tokens_map.json (deflated 42%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/ (stored 0%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/training_args.bin (deflated 51%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/adapter_config.json (deflated 54%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/optimizer.pt (deflated 8%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/README.md (deflated 66%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/scheduler.pt (deflated 57%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/rng_state.pth (deflated 25%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/adapter_model.safetensors (deflated 7%)
  adding: fine_tuned_distilbert_lora/checkpoint-500/trainer_state.json (deflated 83%)
  adding: fine_tuned_distilbert_lora/vocab.txt (deflated 53%)
  adding: fine_tuned_distilbert_lora/tokenizer_config.json (deflated 75%)
  addi